In [1]:
import os
os.chdir(r"\Users\youyo\Desktop\research\CRC02")

In [2]:
import pandas as pd

df = pd.read_csv('phenotype.csv') # read the csv file



In [3]:
useful_data = df[["X_centroid", "Y_centroid","manual_leiden_edges_necrosis_muscle"]] # select the columns that we need
useful_data.head()

,X_centroid,Y_centroid,manual_leiden_edges_necrosis_muscle
0,19407.460317,818.736508,edges
1,19335.283727,825.810848,edges
2,19085.623568,833.024550,edges
3,19357.138060,833.358209,edges
4,19296.168539,835.778090,edges


In [ ]:
useful_data = useful_data[useful_data.manual_leiden_edges_necrosis_muscle != "edges"] # remove the rows that are not useful

In [ ]:
useful_data = useful_data[useful_data.manual_leiden_edges_necrosis_muscle != "excluded"] # remove the rows that are not useful

In [ ]:
useful_data = useful_data[useful_data.manual_leiden_edges_necrosis_muscle != "other immune cells"] # remove the rows that are not useful

In [4]:
useful_data.manual_leiden_edges_necrosis_muscle.unique()

array(['edges', 'ECAD+ cells', 'KI67+ECAD+ cells', 'excluded',
       'CD4 T cells', 'other immune cells', 'CD68+', 'Endothelial cells',
       'T regs', 'necrotic', 'ASMA+ cells', 'CD68+CD163+', 'CD8 T',
       'CD163+', 'B cells', 'muscle'], dtype=object)

In [ ]:
useful_data.manual_leiden_edges_necrosis_muscle.value_counts() # count the number of each cell type

In [ ]:
mapping =  dict((v, i) for i, v in enumerate(useful_data.manual_leiden_edges_necrosis_muscle.unique()))
mapping

In [ ]:
useful_data['manual_leiden_edges_necrosis_muscle'] = useful_data['manual_leiden_edges_necrosis_muscle'].map(mapping)
useful_data.head()

In [ ]:
useful_data.reset_index(drop=True, inplace=True)

In [ ]:
useful_data.manual_leiden_edges_necrosis_muscle.value_counts()

In [ ]:
df_0 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 0]
df_1 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 1]
df_7 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 7]
df_12 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 12]
df_6 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 6]
df_3 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 3]
len(df_0), len(df_1), len(df_7), len(df_12), len(df_6), len(df_3)

In [ ]:
df_2 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 2]
df_4 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 4]
df_5 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 5]
df_8 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 8]
df_9 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 9]
df_10 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 10]
df_11 = useful_data[useful_data.manual_leiden_edges_necrosis_muscle == 11]
len(df_2), len(df_4), len(df_5), len(df_8), len(df_9), len(df_10), len(df_11)

In [ ]:
seed = 10
sample_size = 25000
selected_0_rows = df_0.sample(n=sample_size, random_state=seed)
selected_1_rows = df_1.sample(n=sample_size, random_state=seed)
selected_7_rows = df_7.sample(n=sample_size, random_state=seed)
selected_12_rows = df_12.sample(n=sample_size, random_state=seed)
selected_6_rows = df_6.sample(n=sample_size, random_state=seed)
selected_3_rows = df_3.sample(n=sample_size, random_state=seed)
len(selected_0_rows), len(selected_1_rows), len(selected_7_rows), len(selected_12_rows), len(selected_6_rows), len(selected_3_rows)

In [ ]:
smaller_data = pd.concat([selected_0_rows, selected_1_rows, selected_7_rows, selected_12_rows, selected_6_rows, selected_3_rows, df_2, df_4, df_5, df_8, df_9, df_10, df_11]).reset_index(drop=True)

In [ ]:
smaller_data

In [ ]:
smaller_data.manual_leiden_edges_necrosis_muscle.value_counts()

In [ ]:
labels = smaller_data["manual_leiden_edges_necrosis_muscle"].to_numpy() # get the labels

In [ ]:
len(labels)

In [ ]:
smaller_data

In [ ]:
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt

image = tiff.imread('necrosis_muscle.tif') # read the image and becomes a numpy array
image.shape # check the shape of the image

In [ ]:
image = image.transpose((1, 2, 0)) # change the shape of the image to (height, width, channel)
image.shape

In [ ]:
def get_cell_boundaries(x_cent, y_cent, windowSize): # function to get the cell boundaries
    x_start = max(int(x_cent - windowSize // 2),0)
    x_end = min(x_start + windowSize, image.shape[1]) # get the x boundaries
    x_pair = (x_start, x_end)

    y_start = max(int(y_cent - windowSize // 2),0)
    y_end = min(y_start + windowSize, image.shape[0]) # get the y boundaries
    y_pair = (y_start, y_end)
    return x_pair, y_pair

In [ ]:
import tensorflow as tf
list_of_images = [] # list to store the images
for index, row in smaller_data.iterrows(): # iterate through the rows
    x_pair, y_pair = get_cell_boundaries(row["X_centroid"], row["Y_centroid"], 64) # call the function to get the cell boundaries
    x_start, x_end = x_pair # get the x boundaries
    y_start, y_end = y_pair # get the y boundaries
    crop = image[y_start:y_end, x_start:x_end] # crop the image
    
    if crop.shape != (64, 64, 3):
        # pad the image with zeros to make it 64x64
        image_tensor = tf.convert_to_tensor(crop)
        padded_image_tensor = tf.image.resize_with_pad(image_tensor, 64, 64)
        padded_image = padded_image_tensor.numpy()
        list_of_images.append(padded_image) # append the padded image to the list
    else:
        list_of_images.append(crop) # append the cropped image to the list
        
converted_images = np.array(list_of_images) # convert the list to a numpy array
#check type
# converted_images = converted_images.astype(np.uint8) # convert the data type to uint8


In [ ]:
import h5py

with h5py.File('medium.h5', 'w') as f: # create a h5 file
    f.create_dataset('images', data=converted_images) # store the images
    f.create_dataset('labels', data=labels) # store the labels